# Autoregression

## Baseline AR

As a baseline for how well autoregression should perform, we tried doing autoregression directly on unit sales for each item. For this, we simply used statsmodels' AR model, and trained it on item unit sales. The kaggle test set is 16 days, from August 16, 2017 to August 31, 2017. So, to get an idea of how this model would perform, we first trained it on all the data in the train set except the last 16 days, then predicted those last 16 days. 

### Some prediction results compared to the truth:

<img src="https://favorita-ee379k.github.io/autoregression/graphs/baseline-ar-pred-comparison.png">

The best score we were able to get on Kaggle when running this AR model on the true test set was 0.662, training on a 365 day history instead of all the available train set. As a comparison, the baseline score provided on Kaggle was .911, a score obtained by using the previous year's sales as a prediction. <br>

## Multi-Level AR

Looking at the data, we noticed that on a per store level, the periodicity of the unit sales is much more visible and easier to predict compared to unit sales on a per item level. Many items average very low unit sales, so the signal is much lower resolution and is more sporadic. 

### Some sales totals for stores

<img src="https://favorita-ee379k.github.io/autoregression/graphs/store_sales.png">

### Some sales totals for items

<img src="https://favorita-ee379k.github.io/autoregression/graphs/item_sales.png">

So we hypothesized that we may be missing out on some information about the overall trends by trying to predict item unit sales. To try to alleviate this issue, we decided to split unit sales data into 4 hierarchical categories, and train/predict on them separately. In our data, each item has a specified family and class, so we decided to use that to split the data categorically. In total, there are 4100 unique items, split into 337 unique classes, which are assigned to 33 unique families.
<br>

#### Example of the data available for each item

<img src="https://favorita-ee379k.github.io/autoregression/graphs/item_dataframe.png">

So after processing the data, we have 4 new datasets to train and predict on:
<ol>
    <li><b>Total unit sales per store</b></li>
    <li><b>Proportion of family sales per store</b></li>
    <li><b>Proportion of class sales per family</b></li>
    <li><b>Proportion of item unit sales per class</b></li>
</ol>

Then to transform our predictions back to the original target of unit sales per item, we can simply multiply these 4 predictions.

### Dataset 1: Some unit sales per store predictions

<img src="https://favorita-ee379k.github.io/autoregression/graphs/store_sales_preds.png">

For the next 3 datasets, we were predicting proportions in the range [0, 1], so we trained on a logit transform $log(\frac{p}{1-p})$ of the data. This way, when the predictions are transformed back with $\frac{e^{pred}}{1-e^{pred}}$, they are limited to the (0, 1) range.

### Dataset 2: Some predictions for family proportion of store sales or $\frac{family\_sales}{store\_sales}$

<img src="https://favorita-ee379k.github.io/autoregression/graphs/family_proportion_preds.png">

### Dataset 3: Some predictions for class proportion of family sales or $\frac{class\_sales}{family\_sales}$

<img src="https://favorita-ee379k.github.io/autoregression/graphs/class_proportion_preds.png">

### Dataset 4: Some predictions of item proportion of class sales or $\frac{item\_sales}{class\_sales}$

<img src="https://favorita-ee379k.github.io/autoregression/graphs/item_proportion_predictions.png">

### Putting it all together:

To transform these predictions into a prediction for unit sales per item, we do $store\_sales * \frac{family\_sales}{store\_sales} * \frac{class\_sales}{family\_sales} * \frac{item\_sales}{class\_sales} = item\_sales$

### Some unit sales predictions

<img src="https://favorita-ee379k.github.io/autoregression/graphs/multi-level-ar-pred-comparison.png">

Comparing these plots to the plots from the baseline AR predictions, it is not immediately apparent that the multi-level model is performing better. The multi-level autoregression did better on some items, but worse on others. However, we were able to score a .588 on Kaggle using this method, a \~16% improvement over the .662 that the baseline model scored.

### Comparing the two models (error per sample over 16 day prediction period)

<img src="https://favorita-ee379k.github.io/autoregression/graphs/all-baseline-vs-ml-errors.png">

<img src="https://favorita-ee379k.github.io/autoregression/graphs/ml-errors-minus-baseline-errors.png">

From the above graphs, it is clear to see that the multi-level AR predictions were better on average. The baseline AR predictions have some very high outliers when looking at errors for individual items, some as high as 35. In comparison, the multi-level AR's worst errors for individual items are close to 5. And when taking the difference of errors $ multi\ level\ errors\ per\ sample - baseline\ errors\ per\ sample$, the result is usually negative, meaning the multi-level model tends to have a lower error for a given sample. 

### Can we combine the best predictions from both to get a better prediction?

### Comparing average error per store difference

<img src="https://favorita-ee379k.github.io/autoregression/graphs/ml-errors-minus-baseline-errors-per-store.png">

### Comparing average error per item difference

<img src="https://favorita-ee379k.github.io/autoregression/graphs/ml-errors-minus-baseline-errors-per-item.png">

According to the errors, there are a few stores and some items that the baseline model tends to do better on. Again, these errors are from using the last 16 days of the Kaggle train set as a test set. So we tried using the best store results and the best item results based on these errors. The hope was that if a model did better on a given store or item in our test, it will also do better for that same store or item in the real test.

Unfortunately, in practice, keeping the best predictions based on average per store error gave a .600 and keeping the best predictions based on average per item error gave .594, neither an improvement over the multi-level AR model alone. Since the best store errors and best item errors were only based on one test set, we were probably overfitting to our test set. Comparing many predictions for different places in the training set could probably give a better idea of which model performs best on which store/item.

### Conclusion for autoregression

With autoregression, we were able to significantly beat the baseline score .911 calculated from last years' sales, obtaining a .588 score on Kaggle at best. However, we were getting much better results with other models, so we did not explore autoregression further than this, and instead focused on improving our other models.